In [3]:
from easydict import EasyDict as edict
from dataset import get_dataset
from torch.utils.data import DataLoader

data_args = edict(
    split="train",
    task="vqa",
    dataset="MedXpertQA",
    seed=0,
    image_path="/media/yesindeed/DATADRIVE1/mount/remote_cse/datasets/MedXpertQA",
)

dataset = get_dataset(data_args)
data_loader = DataLoader(dataset, batch_size=1,
                         collate_fn=lambda batch: batch[0])

Logger is not set.


In [4]:
dataset[0]

{'image': [tensor([[[  0,   0,   0,  ...,  21,  17,  18],
           [  0,   0,   0,  ...,  23,  19,  19],
           [  0,   0,   0,  ...,  18,  16,  12],
           ...,
           [  0,   0,   0,  ..., 132, 125, 143],
           [  0,   0,   0,  ..., 140, 133, 146],
           [  0,   0,   0,  ..., 155, 147, 164]],
  
          [[  0,   0,   0,  ...,  21,  17,  18],
           [  0,   0,   0,  ...,  23,  19,  19],
           [  0,   0,   0,  ...,  18,  16,  12],
           ...,
           [  0,   0,   0,  ..., 132, 125, 143],
           [  0,   0,   0,  ..., 140, 133, 146],
           [  0,   0,   0,  ..., 155, 147, 164]],
  
          [[  0,   0,   0,  ...,  21,  17,  18],
           [  0,   0,   0,  ...,  23,  19,  19],
           [  0,   0,   0,  ...,  18,  16,  12],
           ...,
           [  0,   0,   0,  ..., 132, 125, 143],
           [  0,   0,   0,  ..., 140, 133, 146],
           [  0,   0,   0,  ..., 155, 147, 164]]], dtype=torch.uint8),
  tensor([[[114, 102, 101,  ...

In [10]:
for batch in data_loader:
    break

batch["image"]

[tensor([[[  0,   0,   0,  ...,  21,  17,  18],
          [  0,   0,   0,  ...,  23,  19,  19],
          [  0,   0,   0,  ...,  18,  16,  12],
          ...,
          [  0,   0,   0,  ..., 132, 125, 143],
          [  0,   0,   0,  ..., 140, 133, 146],
          [  0,   0,   0,  ..., 155, 147, 164]],
 
         [[  0,   0,   0,  ...,  21,  17,  18],
          [  0,   0,   0,  ...,  23,  19,  19],
          [  0,   0,   0,  ...,  18,  16,  12],
          ...,
          [  0,   0,   0,  ..., 132, 125, 143],
          [  0,   0,   0,  ..., 140, 133, 146],
          [  0,   0,   0,  ..., 155, 147, 164]],
 
         [[  0,   0,   0,  ...,  21,  17,  18],
          [  0,   0,   0,  ...,  23,  19,  19],
          [  0,   0,   0,  ...,  18,  16,  12],
          ...,
          [  0,   0,   0,  ..., 132, 125, 143],
          [  0,   0,   0,  ..., 140, 133, 146],
          [  0,   0,   0,  ..., 155, 147, 164]]], dtype=torch.uint8),
 tensor([[[114, 102, 101,  ..., 170, 175, 172],
          [114,

In [12]:
type(batch["image"]) == list

True